In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
import pylab as plt
from scipy import stats
import xlwings as xw

/Users/matthias/anaconda/lib/python2.7/site-packages/numexpr/cpuinfo.py:76: UserWarning: [Errno 2] No such file or directory
  stacklevel=stacklevel + 1):


In [7]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2017-02-22/"
#path = "/Users/matthias/Desktop/PhD Temp/2017-02-22/"
#path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
files_start = 8005
files_end = 8010

In [8]:
# cup configuration
#cup_config = cycle_Sb
Sn_isotopes = ["112", "114", "115", "116", "117", "118", "119", "120", "122", "124"]
cup_config = cycles2

# Mass Range of cup configuration
mass_range = cycles2_mass_range

# Isotopes used for Interference correction
#corr_isotopes_1 = {"Cd" : "111", "Te" : "125"}
corr_isotopes_2 = {"Cd" : "111", "Te" : "125", "Xe": "129"}
#corr_isotopes_2 = {}
corr_isotopes_Sb = {"Te": "126", "Xe": "129"}

norm_ratio = ["116","120"]
denom_isotope = norm_ratio[1]
Sn_monitor = ["111", "113", "125"]


def eval_iso_list(isotopes_list,norm_ratio, monitor_iso):
    isotope_nom = norm_ratio[0]
    isotope_den = norm_ratio[1]
    isotopes_list.remove(norm_ratio[1])
    for i in range(len(monitor_iso)):
        isotopes_list.append(monitor_iso[i])
    isotopes_list.sort()
    return isotopes_list

isotope_ls = eval_iso_list(Sn_isotopes, norm_ratio, Sn_monitor)

data_sample_column = [(i + "/" + denom_isotope) for i in isotope_ls]    
isotopes = [isotope_ls]

#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = False
# background correction
blk_corr = True

In [9]:
database["Sn"]["Ratios"].get_all_ratios("118")

{'112': 0.04012733280838835,
 '114': 0.02718272796408838,
 '115': 0.013952768411772228,
 '116': 0.6003216970529051,
 '117': 0.3167343038085431,
 '119': 0.3545801449655757,
 '120': 1.346012773661222,
 '122': 0.19124957095842837,
 '124': 0.23903571644894908}

In [10]:

mass_range.get_graph_of_corr(corr_isotopes_2)

{'110': {'111'},
 '111': set(),
 '112': {'111'},
 '113': set(),
 '114': {'111'},
 '115': set(),
 '116': {'111'},
 '117': set(),
 '118': set(),
 '119': set(),
 '120': {'125'},
 '121': set(),
 '122': {'125'},
 '123': {'125'},
 '124': {'125', '129'},
 '125': set(),
 '126': {'129'},
 '127': set(),
 '129': set(),
 '131': set()}

In [11]:
mass_range.get_mass_range()

{'110': ['Pd', 'Cd'],
 '111': ['Cd'],
 '112': ['Sn', 'Cd'],
 '113': ['Cd', 'In'],
 '114': ['Sn', 'Cd'],
 '115': ['Sn', 'In'],
 '116': ['Sn', 'Cd'],
 '117': ['Sn'],
 '118': ['Sn'],
 '119': ['Sn'],
 '120': ['Sn', 'Te'],
 '121': ['Sb'],
 '122': ['Sn', 'Te'],
 '123': ['Sb', 'Te'],
 '124': ['Sn', 'Te', 'Xe'],
 '125': ['Te'],
 '126': ['Te', 'Xe'],
 '127': ['I'],
 '129': ['Xe'],
 '131': ['Xe']}

In [12]:
mass_range.get_order_of_corr(corr_isotopes_2)

[{'111', '113', '115', '117', '118', '119', '121', '125', '127', '129', '131'},
 {'110', '112', '114', '116', '120', '122', '123', '124', '126'}]

In [13]:
columns_1 = ["cycle", "sample", "date", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)"]
columns_2 = ["cycle", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame()
blk_ls = []
sample_ls = []
files_1 = range(files_start, files_end, 1)
files_1 = range(6411,6518,1) + range(6526,6534, 1) + range(6537,6577, 1)

for sample in files_1:
    
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    df_zero = df.data_zero_corr(sample)
    df_zero_1 = pd.DataFrame(df_zero["cycle1"])
    df_zero_2 = pd.DataFrame(df_zero["cycle2"])
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    df_zero_1["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    df_zero_1["sample"] = sample_name
    df_zero_1["cycle"] = df_zero_1.index.values
    df_zero_2["cycle"] = df_zero_2.index.values
    df_zero_1.index = df_zero_1["date"]
    df_zero_1 = df_zero_1[columns_1]
    df_zero_2 = df_zero_2[columns_2]
    df_all = df_all.append(df_zero_1.merge(df_zero_2), ignore_index = True)
    #df_all = df_all.append(df_zero_1, ignore_index = True)
    
    if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
        blk_ls.append(sample)
    elif sample_name == "SQ" or sample_name == "Teflon blk" or sample_name == "4ml HDPE blank":
        None
    else:
        sample_ls.append(sample)

df_all.set_index("date", inplace = True)

#for x in [123]:
#    sample_ls.remove(x)
#blk_ls = blk_ls[0::5]
#blk_ls.remove(5234)
df_all

,cycle,sample,H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),...,H5 (2),H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2)
date,,,,,,,,,,,,,,,,,,,,,
2017-02-23 11:16:00,1,blank sol,0.000583,0.000453,0.004019,0.000794,0.002201,0.000701,0.001614,0.000044,...,0.000093,-1.323650e-05,0.000489,0.000158,0.000362,0.000167,0.003368,0.000619,0.001713,0.000550
2017-02-23 11:16:00,2,blank sol,0.000477,0.000332,0.003219,0.000578,0.001612,0.000493,0.001237,0.000050,...,0.000065,-1.037260e-05,0.000506,0.000140,0.000373,0.000195,0.003595,0.000674,0.001884,0.000607
2017-02-23 11:16:00,3,blank sol,0.000513,0.000366,0.003438,0.000629,0.001770,0.000561,0.001295,0.000020,...,0.000080,1.391000e-05,0.000578,0.000144,0.000382,0.000197,0.003613,0.000672,0.001944,0.000596
2017-02-23 11:16:00,4,blank sol,0.000495,0.000326,0.003372,0.000586,0.001747,0.000554,0.001301,0.000035,...,0.000075,3.533780e-05,0.000473,0.000110,0.000320,0.000145,0.003165,0.000562,0.001546,0.000496
2017-02-23 11:16:00,5,blank sol,0.000564,0.000400,0.003780,0.000757,0.002022,0.000644,0.001451,0.000041,...,0.000060,1.890670e-05,0.000543,0.000136,0.000389,0.000183,0.003555,0.000646,0.001865,0.000564
2017-02-23 11:16:00,6,blank sol,0.000570,0.000398,0.003687,0.000740,0.001983,0.000603,0.001417,0.000024,...,0.000069,-7.519900e-06,0.000494,0.000162,0.000355,0.000153,0.003442,0.000628,0.001769,0.000542
2017-02-23 11:16:00,7,blank sol,0.000540,0.000379,0.003549,0.000684,0.001868,0.000576,0.001375,0.000051,...,0.000067,-2.518900e-06,0.000528,0.000159,0.000366,0.000156,0.003416,0.000650,0.001786,0.000542
2017-02-23 11:16:00,8,blank sol,0.000541,0.000352,0.003519,0.000682,0.001877,0.000608,0.001351,0.000043,...,0.000096,4.623800e-06,0.000607,0.000148,0.000426,0.000164,0.003846,0.000720,0.002070,0.000647
2017-02-23 11:16:00,9,blank sol,0.000635,0.000441,0.003937,0.000807,0.002138,0.000658,0.001530,0.000034,...,0.000112,1.247720e-05,0.000830,0.000116,0.000606,0.000219,0.005172,0.001093,0.003098,0.000946


In [14]:
files_1 = range(files_start, files_end, 1)
df_raw_signals = pd.DataFrame()
df_internal_norm = pd.DataFrame()

for sample in sample_ls:
    df = NU_data_read(path, sample, cup_config)
    cycles = range(1, len(df.data_read(sample).index)+1)
    # arbitrary blank positions
    blk1 = [item for item in blk_ls if item < sample]
    blk2 = [item for item in blk_ls if item > sample]
    blk1 = blk1[-1]
    blk2 = blk2[0]
    blk_corr_sample = True
    
    # blank measurement surrounds sample
    #if ((sample-1) in blk_ls) and ((sample+1) in blk_ls):
    #    blk1 = sample-1
    #    blk2 = sample+1
    #    blk_corr_sample = True
    #else:
    #    blk_corr_sample = False
    
    if (blk_corr == True) and (blk_corr_sample == True):
        blk_1 = NU_data_read(path, blk1, cup_config)
        blk_2 = NU_data_read(path, blk2, cup_config)
        df_zero = df.data_zero_corr(sample)
        df_bgd_1 = blk_1.data_zero_corr(blk1)
        df_bgd_2 = blk_2.data_zero_corr(blk2)   
        
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        df_bgd_1 = new_corr.mad_outlier_rejection_dict(df_bgd_1)
        df_bgd_2 = new_corr.mad_outlier_rejection_dict(df_bgd_2)
        df_bgd_corr = new_corr.data_bgd_corr(df_bgd_1, df_bgd_2)
        new_corr.line2_corr(df_bgd_corr, "119")
    else:
        df_zero = df.data_zero_corr(sample)
        new_corr = evaluation(df_zero, cycles, isotopes, cup_config, database, mass_range, corr_isotopes_2 , denom_corr_ratio)
        new_corr.line2_corr(df_zero, "119")
    
    data_sample_raw = new_corr.raw_signals(denom_isotope)
    data_sample_raw = pd.DataFrame.from_dict(data_sample_raw, orient = 'index')
    data_sample_raw = new_corr.mad_outlier_rejection(data_sample_raw)
    
    data_sample = new_corr.internal_norm_1(norm_ratio, denom_isotope, iter_beta)
    #data_sample = new_corr.raw_ratios_corr(denom_isotope)
    data_sample = pd.DataFrame.from_dict(data_sample, orient = 'index')
    data_sample = new_corr.mad_outlier_rejection(data_sample)
    
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_sample = data_sample.drop(denom_isotope, axis=1)
    data_sample = data_sample[sorted(data_sample.columns)]
    data_sample.columns = data_sample_column
    data_sample["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_sample["sample"] = sample_name
    data_sample["cycle"] = data_sample.index.values
    
    data_sample_raw["cycle"] = data_sample_raw.index.values
    data_sample = data_sample.merge(data_sample_raw)
    
    df_internal_norm = df_internal_norm.append(data_sample)

nu_data_reduction.py:667: RuntimeWarning: divide by zero encountered in divide
  modified_z_score = 0.6745 * diff / med_abs_deviation
nu_data_reduction.py:667: RuntimeWarning: invalid value encountered in divide
  modified_z_score = 0.6745 * diff / med_abs_deviation
nu_data_reduction.py:669: RuntimeWarning: invalid value encountered in greater
  return modified_z_score > thresh


In [15]:
df_internal_norm= df_internal_norm.set_index("date")
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_internal_sub = pd.DataFrame(df_internal_norm, columns = columns)

df_internal_sub["Cd/Sn"] = (df_internal_norm["111"]/12.722)/(df_internal_norm["118"]/24.223)
df_internal_sub["Te/Sn"] = (df_internal_norm["125"]/7.0509)/(df_internal_norm["118"]/24.223)

#df_internal_sub["Te/Sn"] = (df_internal_norm["126"]/18.8066)/(df_internal_norm["118"]/24.223)
#df_internal_sub = df_internal_sub[df_internal_sub["118"] >= 7]
#df_external_sub.to_csv(path_sav + "Sn_H9_L4_internal_corr_all.csv")
df_internal_sub["124/120"]

date
2017-02-23 11:27:00    0.177590
2017-02-23 11:27:00    0.177624
2017-02-23 11:27:00    0.177573
2017-02-23 11:27:00    0.177617
2017-02-23 11:27:00    0.177609
2017-02-23 11:27:00    0.177687
2017-02-23 11:27:00    0.177546
2017-02-23 11:27:00    0.177566
2017-02-23 11:27:00    0.177599
2017-02-23 11:27:00    0.177621
2017-02-23 11:27:00    0.177622
2017-02-23 11:27:00    0.177483
2017-02-23 11:27:00    0.177640
2017-02-23 11:27:00    0.177578
2017-02-23 11:27:00    0.177609
2017-02-23 11:27:00    0.177606
2017-02-23 11:27:00    0.177591
2017-02-23 11:27:00    0.177554
2017-02-23 11:27:00    0.177564
2017-02-23 11:27:00    0.177542
2017-02-23 11:52:00    0.177496
2017-02-23 11:52:00    0.177537
2017-02-23 11:52:00    0.177622
2017-02-23 11:52:00    0.177670
2017-02-23 11:52:00    0.177596
2017-02-23 11:52:00    0.177567
2017-02-23 11:52:00    0.177605
2017-02-23 11:52:00    0.177555
2017-02-23 11:52:00    0.177563
2017-02-23 11:52:00    0.177567
                         ...   
201

In [16]:
def SE(x):
    SE = np.std(x) / x.count()
    return SE

def SD2(x):
    SD2 = 2 * np.std(x)
    return SD2

df_internal_norm_mean = df_internal_sub.drop("cycle", axis=1).groupby(df_internal_sub.index).mean()
df_internal_norm_mean_sd = df_internal_sub.drop("cycle", axis=1).groupby(df_internal_sub.index).agg([np.mean, SD2])
df_internal_norm_mean["sample"] = df_internal_sub[~df_internal_sub.index.duplicated(keep='first')]["sample"]
df_internal_norm_mean_sd["sample"] = df_internal_sub[~df_internal_sub.index.duplicated(keep='first')]["sample"]
#df_internal_norm_mean_sd = df_internal_norm_mean_sd[1:8]
#df_internal_norm_mean = df_internal_norm_mean[df_internal_norm_mean["118"] >= 7]
df_internal_norm_mean["124/120"]
#epsilon = epsilon.drop(["118"], axis = 1)
#plotplot = df_internal_norm_mean[(df_internal_norm_mean["125"] < 0.00010)  & (df_internal_norm_mean["124/120"] > 0.17745)] 
#plot(plotplot["125"], plotplot["124/120"], "ro")

date
2017-02-23 11:27:00    0.177591
2017-02-23 11:52:00    0.177574
2017-02-23 12:07:00    0.177590
2017-02-23 12:19:00    0.177559
2017-02-23 12:45:00    0.177589
2017-02-23 12:59:00    0.177627
2017-02-23 13:12:00    0.177586
2017-02-23 13:37:00    0.177592
2017-02-23 13:51:00    0.177558
2017-02-23 14:04:00    0.177605
2017-02-23 14:29:00    0.177615
2017-02-23 14:45:00    0.177579
2017-02-23 14:57:00    0.177591
2017-02-23 15:23:00    0.177572
2017-02-23 15:37:00    0.177664
2017-02-23 15:50:00    0.177576
2017-02-23 16:15:00    0.177597
2017-02-23 16:30:00    0.177575
2017-02-23 16:43:00    0.177554
2017-02-23 17:08:00    0.177580
2017-02-23 17:22:00    0.177590
2017-02-23 17:35:00    0.177596
2017-02-23 18:00:00    0.177647
2017-02-23 18:15:00    0.177626
2017-02-23 18:27:00    0.177615
2017-02-23 18:53:00    0.177631
2017-02-23 19:07:00    0.177612
2017-02-23 19:20:00    0.177617
2017-02-23 19:45:00    0.177579
2017-02-23 20:00:00    0.177566
                         ...   
201

In [17]:
df_internal_norm_sample_sd = df_internal_norm_mean.groupby(df_internal_norm_mean["sample"]).agg([np.mean, SD2])
df_internal_norm_sample_sd

111/120                 112/120            \
                              mean           SD2      mean       SD2   
sample                                                                 
J.W. Leach 1b             0.000230  0.000000e+00  0.029827  0.000000   
J.W. Leach 2a             0.000052  0.000000e+00  0.029807  0.000000   
J.W. Leach 5b             0.000070  0.000000e+00  0.029816  0.000000   
NIST 40ppb                0.000003  7.638607e-06  0.029823  0.000011   
NIST 40ppb + 0.002ppb U   0.000003  2.817828e-06  0.029826  0.000005   
NIST 40ppb + 0.1ppb Zr    0.000005  4.005000e-07  0.029819  0.000001   
NIST 40ppb + 0.2ppb Ru    0.000005  2.687251e-06  0.029820  0.000005   
NIST 40ppb + 0.5ppb Pd    0.000002  0.000000e+00  0.029819  0.000000   
NIST 40ppb + 2.5ppb Mo    0.000003  2.096680e-06  0.029823  0.000004   
NIST 5.5ppb               0.000010  3.868951e-05  0.029828  0.000068   
NIST 5.5ppb + 0.024 Te    0.000027  0.000000e+00  0.029789  0.000000   
NIST 5.5ppb + 0.1ppb Ru   0.000002  3.391370e-07  0.029825  0.000004   
NIST 5.5ppb + 0.3ppb Mo  -0.000028  0.000000e+00  0.029869  0.000000   
NIST 5.5ppb + Te         -0.000020  0.000000e+00  0.029845  0.000000   
NIST 9ppb                 0.000007  3.184719e-05  0.029825  0.000052   
NIST 9ppb + 0.05ppb Te    0.000043  0.000000e+00  0.029795  0.000000   
NIST 9ppb +0.1ppb Ru      0.000020  0.000000e+00  0.029803  0.000000   
NIST 9ppb +0.3ppb Mo      0.000009  6.937472e-06  0.029836  0.000022   
ZH09-05 23.3_3b_3 35ppb   0.000035  0.000000e+00  0.029824  0.000000   
ZH09-05 23.3_3b_3 5.5ppb -0.000009  0.000000e+00  0.029860  0.000000   
ZH09-05 23.3_3b_3 9ppb    0.000043  0.000000e+00  0.029820  0.000000   

                           113/120                 114/120                \
                              mean           SD2      mean           SD2   
sample                                                                     
J.W. Leach 1b             0.000244  0.000000e+00  0.020187  0.000000e+00   
J.W. Leach 2a             0.000049  0.000000e+00  0.020189  0.000000e+00   
J.W. Leach 5b             0.000072  0.000000e+00  0.020187  0.000000e+00   
NIST 40ppb                0.000005  5.983377e-06  0.020194  1.220592e-05   
NIST 40ppb + 0.002ppb U   0.000009  5.761370e-06  0.020196  4.935669e-07   
NIST 40ppb + 0.1ppb Zr    0.000008  6.992201e-08  0.020189  5.161443e-06   
NIST 40ppb + 0.2ppb Ru    0.000005  3.503235e-06  0.020188  8.969302e-06   
NIST 40ppb + 0.5ppb Pd    0.000004  0.000000e+00  0.020189  0.000000e+00   
NIST 40ppb + 2.5ppb Mo    0.000005  1.902245e-06  0.020195  2.496826e-07   
NIST 5.5ppb               0.000017  4.723852e-05  0.020198  8.346653e-05   
NIST 5.5ppb + 0.024 Te    0.000003  0.000000e+00  0.020090  0.000000e+00   
NIST 5.5ppb + 0.1ppb Ru   0.000030  9.198049e-06  0.020199  7.388766e-05   
NIST 5.5ppb + 0.3ppb Mo   0.000020  0.000000e+00  0.020256  0.000000e+00   
NIST 5.5ppb + Te         -0.000014  0.000000e+00  0.020212  0.000000e+00   
NIST 9ppb                 0.000010  2.814694e-05  0.020192  5.476295e-05   
NIST 9ppb + 0.05ppb Te    0.000008  0.000000e+00  0.020153  0.000000e+00   
NIST 9ppb +0.1ppb Ru      0.000034  0.000000e+00  0.020178  0.000000e+00   
NIST 9ppb +0.3ppb Mo      0.000010  7.169564e-06  0.020219  1.439102e-05   
ZH09-05 23.3_3b_3 35ppb   0.000038  0.000000e+00  0.020195  0.000000e+00   
ZH09-05 23.3_3b_3 5.5ppb  0.000054  0.000000e+00  0.020273  0.000000e+00   
ZH09-05 23.3_3b_3 9ppb    0.000063  0.000000e+00  0.020191  0.000000e+00   

                           115/120                  ...          122  \
                              mean           SD2    ...         mean   
sample                                              ...                
J.W. Leach 1b             0.010369  0.000000e+00    ...     0.085838   
J.W. Leach 2a             0.010379  0.000000e+00    ...     0.128592   
J.W. Leach 5b             0.010357  0.000000e+00    ...     0.719148   
NIST 40ppb                0.010360  4.188

In [66]:
#df_internal_norm_mean.describe()

In [ ]:
# variation of standard data + mean + 2SD
sel_ratio = "124/118"
plot_Nist = df_internal_norm_mean[df_internal_norm_mean["sample"] == "NIST 200ppb"]
plot_Nist = plot_Nist[(plot_Nist["118"] >= .5)]
#dataplot = plt.plot(plot_Nist["Te/Sn"], plot_Nist["124/118"], "ro")
plt.plot(plot_Nist.index, plot_Nist[sel_ratio], linestyle='-', marker='o', color='b')
plt.plot(plot_Nist.index, pd.rolling_mean(plot_Nist[sel_ratio], 2), linestyle='-', color='r')
plt.axhline(plot_Nist[sel_ratio].mean(), color='r', linestyle='-')
plt.axhline(plot_Nist[sel_ratio].mean()+ 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
plt.axhline(plot_Nist[sel_ratio].mean()- 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
#dataplot = plt.plot(plot_Nist.index, plot_Nist["120"], "ro")
plt.show()

In [ ]:
# variation of standard data + meadian + 25% quantile + 75% quantile
sel_ratio = "120/118"
plot_Nist = df_internal_norm_mean[df_internal_norm_mean["sample"] == "NIST 200ppb"]
plot_Nist = plot_Nist[(plot_Nist["118"] >= .5)]
#dataplot = plt.plot(plot_Nist["Te/Sn"], plot_Nist["124/118"], "ro")
plt.plot(plot_Nist.index, plot_Nist[sel_ratio], linestyle='-', marker='o', color='b')
plt.plot(plot_Nist.index, pd.rolling_mean(plot_Nist[sel_ratio], 2), linestyle='-', color='r')
plt.axhline(plot_Nist[sel_ratio].median(), color='r', linestyle='-')
plt.axhline(plot_Nist[sel_ratio].quantile(0.25), color='r', linestyle='--')
plt.axhline(plot_Nist[sel_ratio].quantile(0.75), color='r', linestyle='--')
#dataplot = plt.plot(plot_Nist.index, plot_Nist["120"], "ro")
plt.show()

In [ ]:
# outlier corr variation of standard data + meadian + 25% quantile + 75% quantile
sel_ratio = "120/118"
plot_Nist = df_internal_norm_mean[df_internal_norm_mean["sample"] == "NIST 200ppb"]
plot_Nist = new_corr.mad_outlier_rejection(plot_Nist.drop("sample", axis=1))
plot_Nist = plot_Nist[(plot_Nist["118"] >= .5)]
#dataplot = plt.plot(plot_Nist["Te/Sn"], plot_Nist["124/118"], "ro")
plt.plot(plot_Nist.index, plot_Nist[sel_ratio], linestyle='-', marker='o', color='b')
plt.plot(plot_Nist.index, pd.rolling_mean(plot_Nist[sel_ratio], 2), linestyle='-', color='r')
plt.axhline(plot_Nist[sel_ratio].mean(), color='r', linestyle='-')
plt.axhline(plot_Nist[sel_ratio].mean()+ 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
plt.axhline(plot_Nist[sel_ratio].mean()- 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
#dataplot = plt.plot(plot_Nist.index, plot_Nist["120"], "ro")
plt.show()

In [ ]:
std_used = "NIST 200ppb"
sub_index = df_internal_norm_mean[(df_internal_norm_mean["sample"].shift(1) == df_internal_norm_mean["sample"].shift(-1)) & (df_internal_norm_mean["sample"].shift(1) == std_used)].index
#corr_values = df_internal_norm_mean[(df_external_norm_mean >= 0.74289)]
#corr_values = df_internal_norm_mean[:"20150110 10:00"]
corr_values = df_internal_norm_mean[df_internal_norm_mean["118"] >= 0.1]
corr_values_2 = corr_values[corr_values["sample"] == std_used]
#corr_values = df_internal_norm_mean
sample = corr_values["sample"]
sample_2 = corr_values_2["sample"]
std_mean = corr_values[(corr_values["sample"] == std_used)].mean()
new = corr_values[corr_values.columns[corr_values.dtypes != 'object']]
new_2 = corr_values_2[corr_values_2.columns[corr_values_2.dtypes != 'object']]

#std_mean["112/120", "114/120", "115/120", "117/120", "118/120", "119/120", "122/120", "124/120"] = [0.029823, 0.020189, 0.010372, 0.235299, 0.742923, 0.263443, 0.142078, 0.177546]
#print std_mean
#epsilon =  ((new / std_mean)-1)*10000
#new["sample"] = sample
epsilon_NIST =  ((new / ((new.shift(1) + new.shift(-1))/2))-1)*10000
epsilon_2 = ((new_2 / ((new_2.shift(1) + new_2.shift(-1))/2))-1)*10000
epsilon_NIST["sample"] = sample
epsilon_2["sample"] = sample

epsilon_NIST = epsilon_NIST[new.index.isin(sub_index)]
epsilon_NIST= epsilon_NIST[epsilon_NIST["sample"] != std_used]
epsilon_2 = epsilon_2[epsilon_2["sample"] == std_used]
epsilon_comp = epsilon_NIST.append(epsilon_2, ignore_index=False)
#epsilon_comp.index = epsilon_comp["Date"]
#epsilon_comp.columns
epsilon_comp = epsilon_comp.sort_index(axis=0)

In [ ]:
#epsilon_comp = epsilon_comp.drop([datetime.strptime("2017-02-24 15:18", "%Y-%m-%d %H:%M")])
epsilon_comp

In [ ]:
def SD2(x):
    SD2 = np.nanstd(x, ddof=1) * 2
    return SD2

epsilon = epsilon_comp.drop(["111", "126", "112", "114", "115", "116", "117", "118", "119", "120", "122", "124", "Cd/Sn", "Te/Sn"], axis = 1)
#epsilon = epsilon.drop(["118"], axis = 1)
epsilon_sum = np.round(epsilon.groupby("sample").agg([np.mean, SD2]), decimals=2)
epsilon_sum

In [ ]:
### sel_ratio = "112/118"
std = "NIST 100ppb"

def plot_epsilon(sel_ratio, std, xlim):
    x = range(1, len(epsilon_sum.index)+1)
    plt.errorbar(epsilon_sum[sel_ratio]["mean"], x, xerr = epsilon_sum[sel_ratio]["SD2"], ls='none', marker="o")
    plt.title("samples" + ", 2SD = " + str(np.round((2*epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].std()),2)) + ", n= " +  str((epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].count())))
    plt.xlim(xlim)
    plt.ylim([0.5, len(epsilon_sum.index)+1])
    plt.yticks(x, epsilon_sum.index, size='small', rotation=0)
    plt.axvline(epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].mean(), color='k', linestyle='-')
    plt.axvline(epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].mean()+ 2*epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].std(), color='r', linestyle='--')
    plt.axvline(epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].mean()- 2*epsilon_comp[epsilon_comp["sample"]== std][sel_ratio].std(), color='r', linestyle='--')
    #plt.axvline(plot_Nist[sel_ratio].mean()- 2*plot_Nist[sel_ratio].std(), color='r', linestyle='--')
    plt.ylabel(r'$\mathregular{\epsilon^{'+sel_ratio+'}}$Sn$_{NIST}$')
    #savefig(path+"epsilon_118.pdf", bbox_inches='tight')

In [ ]:
wb = xw.books.add()
#wb.sheets.add()
wb.sheets["Sheet1"].range('A1').value = df_all
df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).mean()
df_all_mean["sample"] = df_all[~df_all.index.duplicated(keep='first')]["sample"]
wb.sheets["Sheet1"].range('A344').value = df_all_mean

In [ ]:
#wb.sheets.add()
wb.sheets[3].range('AF1').value = df_internal_sub
#df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).mean()
#df_all_mean["sample"] = df_all[~df_all.index.duplicated(keep='first')]["sample"]
#xw.Range('A344').value = df_all_mean

In [ ]:
#wb.sheets.add()
wb.sheets[4].range('A21').value = df_internal_norm_mean_sd

In [ ]:
wb.sheets[4].range('BF21').value = epsilon_comp

In [ ]:
#epsilon_comp[epsilon_comp["sample"]== "NIST 200ppb"]["117/118"].std()
wb.sheets[4].range('BF36').value = epsilon_sum

In [ ]:
#sel_ratio = ["116/118", "117/118", "119/118", "120/118", "122/118", "124/118"]
#sel_ratio = ["116/117", "118/117", "119/117", "120/117", "122/117", "124/117"]
sel_ratio = ["116/120", "117/120", "118/120", "119/120", "122/120", "124/120"]
#sel_ratio = ["112/120", "114/120", "115/120"]
#sel_ratio = ["112/118", "114/118", "115/118"]
xlim = [-1, 1]

fig = plt.figure(figsize=(10,15))

for i in range(len(sel_ratio)):
    ax = plt.subplot(5,2,i+1)
    plot_epsilon(sel_ratio[i], "NIST 200ppb", xlim)
fig.tight_layout()
#wb.sheets[4].pictures.add(fig, name= '116/120 norm - bgd_corr 1', update = True)
#savefig(path+"epsilon_x_bgd_118_2_100ppb.pdf", bbox_inches='tight')

In [ ]:
sel_ratio = "116/118"
epsilon_comp = epsilon_comp[epsilon_comp["sample"] == "NIST 200ppb"]
x = epsilon_comp.index
plt.plot(x, epsilon_comp[sel_ratio], marker="o")
plt.ylim([-0.5, 0.5])
#plt.xticks(x, epsilon_comp["sample"], size='small', rotation=90)
plt.axhline(epsilon_comp[sel_ratio].mean(), color='k', linestyle='-')
plt.axhline(epsilon_comp[sel_ratio].mean()+ 2*epsilon_comp[sel_ratio].std(), color='r', linestyle='--')
plt.axhline(epsilon_comp[sel_ratio].mean()- 2*epsilon_comp[sel_ratio].std(), color='r', linestyle='--')
plt.ylabel(r'$\mathregular{\epsilon^{118}}$Sn')
#savefig(path+"epsilon_119.pdf", bbox_inches='tight')

In [76]:
df_all.to_csv(path + "Sn_H8_L4_2_cycles_NU_Signals_zero_corr.csv", mode='w', header=True)

In [18]:
df_internal_norm.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_116_120_bgd_corr_outlier_corr_all_2.csv", mode='w', header=True)

In [19]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_116_120_bgd_corr_outlier_corr_2.csv", mode='w', header=True)

In [ ]:
df_std.to_csv(path_sav + "All_standards_internal_norm_mean.csv")

In [ ]:
df_all_mean_1 = pd.read_csv(path_sav + "Sn_H9_L4_1_cycles_internal_norm_mean.csv")

In [ ]:
df_all_mean_2 = pd.read_csv(path_sav + "Sn_H8_L4_2_cycles_internal_norm_mean.csv")

In [ ]:
df_mean_all = df_all_mean_1.append(df_all_mean_2)

In [ ]:
#old Cd/Sn & Te/Sn extraction - doesn't include bgd corr
df_internal_norm= df_internal_norm.set_index("date")
columns = ["cycle", "sample"]
columns.extend(data_sample_column)
columns.extend(sorted(data_sample_raw.drop("cycle", axis=1).columns))
df_internal_sub = pd.DataFrame(df_internal_norm, columns = columns)

df_all_sub = df_all[(df_all["sample"] != "blank sol") & (df_all["sample"] != "SQ") & (df_all["sample"] != "teflon blank")]
df_internal_sub["111"] = df_all_sub["L3 (1)"]
df_internal_sub["125"] = df_all_sub["H5 (2)"]
df_internal_sub["Cd/Sn"] = (df_all_sub["L3 (1)"]/12.722)/(df_all_sub["H4 (1)"]/24.223)
df_internal_sub["Te/Sn"] = (df_all_sub["H5 (2)"]/7.0509)/(df_all_sub["H4 (1)"]/24.223)
#df_external_sub.to_csv(path_sav + "Sn_H9_L4_internal_corr_all.csv")

In [ ]:
df_mean_all = df_mean_all[2:]
df_mean_all.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
df_mean_all = df_mean_all[["Date", "112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120", "118","sample"]]
df_mean_all = df_mean_all.set_index("Date")

In [ ]:
df_std = df_mean_all[(df_mean_all["sample"] == "SPEX1 100ppb") | (df_mean_all["sample"] == "SPEX 100ppb") | (df_mean_all["sample"] == "NIST 100ppb")]
df_std

In [ ]:
df_std[["112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120", "118"]] = df_std[["112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120", "118"]].astype(float)
df_std.index = df_std.index.astype(datetime64)
df_std = df_std[(df_std.index < pd.Timestamp('2015-03-10')) | (df_std.index > pd.Timestamp('2015-03-12'))]
df_std = df_std[(df_std.index < pd.Timestamp('2014-11-29 18:25:00')) | (df_std.index > pd.Timestamp('2014-11-29 20:27:00'))]
df_std

In [ ]:
df_std = df_std[(df_std["118"] > 5) & (df_std["118/120"] > 0.7428)]
for i in df_std.index[df_std["sample"] == "SPEX1 100ppb"]:
    df_std.ix[i, "sample"] = "SPEX 100ppb"

In [ ]:
sample = df_std["sample"]
std_mean = df_std[(df_std["sample"] == "SPEX 100ppb")].mean()
new = df_std[df_std.columns[df_std.dtypes != 'object']]
epsilon =  ((new / std_mean)-1)*10000
epsilon["sample"] = sample
epsilon
epsilon[(epsilon["118/120"] > 0.4) | (epsilon["118/120"] < -0.4)]

In [ ]:
lee = pd.DataFrame({"112/120" : pd.Series(0.029812), "114/120" : pd.Series(0.020195), "115/120": pd.Series(0.010366), "116/120": pd.Series(0.4460), "117/120" : pd.Series(0.235313), "118/120" : pd.Series(0.742935), "119/120" : pd.Series(0.263430), "122/120" : pd.Series(0.142086), "124/120" : pd.Series(0.177588)})
epsilon_lee = ((lee / std_mean[["112/120", "114/120", "115/120", "116/120", "117/120", "118/120", "119/120", "122/120", "124/120"]])-1)*10000
epsilon_lee["sample"] = "Lee et al. (1995)"
epsilon_lee